<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [2]:
import pandas as pd
import numpy as np
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score

import optuna
from optuna.visualization import plot_optimization_history

Importering av relevant data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
mw_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                      header=0,
                      sep=',',
                      index_col=0)

# Formatterer index til riktig format og datatype
mw_data.index = pd.to_datetime(mw_data.index,
                               format='%Y-%m-%d %H:%M:%S')

Mounted at /content/drive


Tilfeldighetsfrø

In [4]:
random_seed = 123

Beskrivelse av datasett

In [5]:
mw_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT07           29136 non-null  float64
 11  TT08           29136 non-null  float64
 12  PT03           29136 non-null  float64
 13  TT20           29136 non-null  float64
 14  TT09           29136 non-null  float64
 15  TT12           29136 non-null  float64
 16  Mw             460 non-null    float64
dtypes: float64(11),

# Oppdeling av datasett for trening og testing

### Alternativ 1: Fordelt på Enzymtyper

In [ ]:
 def trening_testsett_oppdeling_enzym(df, test_size=0.2, random_seed=123):

  # Responsvariabel
  responsvariabel = df.columns[-1]

  # Deler opp markert og umarkert del av datasettet
  data_markert = df.dropna(subset=[responsvariabel])
  data_umarkert = df[df[responsvariabel].isna()]

  # Legger til kategorisk variabel for enzymtype til fordeling av datsettet
  data_markert['EnzymType'] = data_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')

  # Splitter datasett i dataasett og kategorisk enzymtype
  enzymtyper = data_markert['EnzymType']
  data_markert.drop(columns=['EnzymType'], inplace=True)

  # Splitter trening og testdata etter enzymtype
  trening_markert, test_markert, _, _ = train_test_split(data_markert,
                                                          enzymtyper,
                                                          test_size=test_size,
                                                          stratify=enzymtyper,
                                                          random_state=random_seed)

  # Definerer trening og testsett for data
  testdata = test_markert

  felles_indeks = df.index.intersection(testdata.index)
  treningsdata = df.drop(felles_indeks)

  return treningsdata, testdata

## Alternativ 2: Fordelt på dag og kontinuerlige tidsserier

In [10]:
def trening_testsett_oppdeling_dag(df, andel=0.2):

    # Responsvariabel
    responsvariabel = df.columns[-1]

    # Lager nytt trening og testsett
    data_markert = df.dropna(subset=[responsvariabel])

    trening_sett = copy.deepcopy(data_markert)
    test_sett = pd.DataFrame(columns=list(data_markert.columns))

    # Går gjennom hver dag, hver uke og samler første andel av rader til testsett
    for uke in data_markert.index.isocalendar().week.unique():
        for dag in data_markert.index.isocalendar().day.unique():
            data = data_markert[(data_markert.index.isocalendar().week == uke) &
                      (data_markert.index.isocalendar().day == dag)]
            test_rader = data.shape[0] * andel
            test_data_dag = data.head(round(test_rader))

            test_sett = pd.concat([test_sett, test_data_dag])
            trening_sett.drop(test_data_dag.index, inplace=True)

    return trening_sett, test_sett

# NIR målinger inkludert

## Oppdeling av trening og testsett

### Alternativ 1: Fordelt på Enzymtype

In [ ]:
mw_trening, mw_test = trening_testsett_oppdeling_enzym(mw_data)

In [ ]:
X_trening = mw_trening.iloc[:, :-1]
X_test = mw_test.iloc[:, :-1]
y_trening = mw_trening.iloc[:, -1]
y_test = mw_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)

  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-09 12:02:22,074] A new study created in memory with name: no-name-94b6287d-2c48-4870-88fb-68732daebe0d
[I 2024-02-09 12:02:22,607] Trial 0 finished with value: 343061.65603100054 and parameters: {'n_estimators': 130, 'max_depth': 6}. Best is trial 0 with value: 343061.65603100054.
[I 2024-02-09 12:02:23,087] Trial 1 finished with value: 1000148.6252579209 and parameters: {'n_estimators': 131, 'max_depth': 3}. Best is trial 0 with value: 343061.65603100054.
[I 2024-02-09 12:02:23,468] Trial 2 finished with value: 451476.08731826267 and parameters: {'n_estimators': 118, 'max_depth': 5}. Best is trial 0 with value: 343061.65603100054.
[I 2024-02-09 12:02:23,915] Trial 3 finished with value: 619703.0687058389 and parameters: {'n_estimators': 192, 'max_depth': 4}. Best is trial 0 with value: 343061.65603100054.
[I 2024-02-09 12:02:24,270] Trial 4 finished with value: 1543052.8090622982 and parameters: {'n_estimators': 195, 'max_depth': 2}. Best is trial 0 with value: 343061.65603

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  y_trening_prediksjon = rf_pipeline.predict(X_trening)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 119, 'max_depth': 9}
Test: MSE: 304150.56270906975, og R^2: 0.9460160945821272
Trening: MSE: 61240.625341091145, og R^2: 0.9893721909646818


### Alternativ 2

In [11]:
mw_trening2, mw_test2 = trening_testsett_oppdeling_dag(mw_data)

In [27]:
X_trening = mw_trening2.iloc[:, :-1]
X_test = mw_test2.iloc[:, :-1]
y_trening = mw_trening2.iloc[:, -1]
y_test = mw_test2.iloc[:, -1]

**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [28]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)

  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-11 12:35:26,678] A new study created in memory with name: no-name-dd8744ca-54d0-4432-b38b-84dd44670ee0
[I 2024-02-11 12:35:27,351] Trial 0 finished with value: 596736.4734090809 and parameters: {'n_estimators': 153, 'max_depth': 6}. Best is trial 0 with value: 596736.4734090809.
[I 2024-02-11 12:35:27,981] Trial 1 finished with value: 570167.3672472985 and parameters: {'n_estimators': 132, 'max_depth': 9}. Best is trial 1 with value: 570167.3672472985.
[I 2024-02-11 12:35:28,533] Trial 2 finished with value: 588210.217103757 and parameters: {'n_estimators': 124, 'max_depth': 7}. Best is trial 1 with value: 570167.3672472985.
[I 2024-02-11 12:35:29,317] Trial 3 finished with value: 557311.0430679698 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 3 with value: 557311.0430679698.
[I 2024-02-11 12:35:29,702] Trial 4 finished with value: 680515.5428640008 and parameters: {'n_estimators': 124, 'max_depth': 5}. Best is trial 3 with value: 557311.0430679698.
[

Evauleringer av beste modell

In [14]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  y_trening_prediksjon = rf_pipeline.predict(X_trening)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 200, 'max_depth': 8}
Test: MSE: 543167.4925566711, og R^2: 0.8849458029028052
Trening: MSE: 67176.19120422158, og R^2: 0.988671792849692


# NIR målinger ekskludert

In [ ]:
mw_data_uten_NIR = mw_data.drop(columns=['NIRfat', 'NIRwater', 'NIRash'])
mw_data_uten_NIR.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   TT07           29136 non-null  float64
 8   TT08           29136 non-null  float64
 9   PT03           29136 non-null  float64
 10  TT20           29136 non-null  float64
 11  TT09           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 4.3 MB


## Oppdeling av trening og testsett

### Alternativ 1: Fordelt på Enzymtyper

In [ ]:
mw_trening, mw_test = oppdeling_alternativ_1(mw_data_uten_NIR)
mw_trening_markert = mw_trening.dropna(subset=["Mw"])

X_trening = mw_trening_markert.iloc[:, :-1]
X_test = mw_test.iloc[:, :-1]
y_trening = mw_trening_markert.iloc[:, -1]
y_test = mw_test.iloc[:, -1]

<ipython-input-6-fb7e234f3b43>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-fb7e234f3b43>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**RandomForestRegressor**

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)

  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-09 12:03:05,190] A new study created in memory with name: no-name-229b8181-0342-4f68-a253-66e2493ca958
[I 2024-02-09 12:03:05,467] Trial 0 finished with value: 935616.6062918743 and parameters: {'n_estimators': 128, 'max_depth': 5}. Best is trial 0 with value: 935616.6062918743.
[I 2024-02-09 12:03:05,841] Trial 1 finished with value: 706656.0612734959 and parameters: {'n_estimators': 156, 'max_depth': 7}. Best is trial 1 with value: 706656.0612734959.
[I 2024-02-09 12:03:06,160] Trial 2 finished with value: 653915.8686557718 and parameters: {'n_estimators': 117, 'max_depth': 10}. Best is trial 2 with value: 653915.8686557718.
[I 2024-02-09 12:03:06,453] Trial 3 finished with value: 678637.2302468478 and parameters: {'n_estimators': 103, 'max_depth': 9}. Best is trial 2 with value: 653915.8686557718.
[I 2024-02-09 12:03:06,971] Trial 4 finished with value: 678556.2210748278 and parameters: {'n_estimators': 195, 'max_depth': 9}. Best is trial 2 with value: 653915.8686557718.


In [ ]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  y_trening_prediksjon = rf_pipeline.predict(X_trening)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 119, 'max_depth': 10}
Test: MSE: 651671.3204451383, og R^2: 0.884334381586855
Trening: MSE: 90005.84287004697, og R^2: 0.9843802230176789
